In [1]:
import numpy as np

def homing_nn_TD(n_trials,learning_rate,eps,gamma):

    # Solving homing task with on-policy TD (SARSA)

    n_steps = 50

    ## Definition of the environment
    N = 7                               #height of the gridworld ---> number of rows
    M = 7                               #length of the gridworld ---> number of columns
    N_states = N * M                     #total number of states
    states_matrix = np.eye(N_states)
    N_actions = 4                                           #number of possible actions in each state: 1->N 2->E 3->S 4->W
    action_row_change = np.array([-1,0,+1,0])               #number of cell shifted in vertical as a function of the action
    action_col_change = np.array([0,+1,0,-1])               #number of cell shifted in horizontal as a function of the action
    End = np.array([1, 1])                                  #terminal state--->reward
    s_end = np.ravel_multi_index(End,dims=(N,M),order='F')  #terminal state. Conversion in single index

    ## Parameters of the model
    #gamma = 0.9                        #discounting factor
                                        #high value is better but it cost more time to decrese steps in the begining
    #learning_rate = 0.3                #constant step-size parameter (learning rate), high value is better
    #eps = 0.5                          #epsilon-greedy SARSA
    
    ## Rewards
    R = 10                              #only when the robot reaches the charger, sited in End state

    ## Variables
    weights = np.random.rand(N_actions,N_states)  
    learning_curve = np.zeros((1,n_trials))

    ## SARSA

    # Start trials
    for trial in range(n_trials):

        # Initialization
        Start = np.array([np.random.randint(N),np.random.randint(M)])   #random start
        s_start = np.ravel_multi_index(Start,dims=(N,M),order='F')      #conversion in single index
        state = Start                                                   #set current state
        s_index = s_start                                               #conversion in single index
        step = 0
       
        # Start steps
        while s_index != s_end and step <= n_steps:

            step += 1
            learning_curve[0,trial] = step

            input_vector = states_matrix[:,s_index].reshape(N_states,1) #convert the state into an input vector 12 by 1

            #compute Qvalues. Qvalue=logsig(weights*input). Qvalue is 2x1, one value for each output neuron
            Q = 1 / ( 1 + np.exp( - weights.dot(input_vector)))    #Qvalue is 2x1 implementation of logsig(4(N_actions) by 1)

            #eps-greedy policy implementation
            greedy = (np.random.rand() > eps)               #1--->greedy action 0--->non-greedy action
            if greedy:
                action = np.argmax(Q)                           #pick best action
            else:
                action = np.random.randint(N_actions)           #pick random action

            state_new = np.array([0,0])
            #move into a new state
            state_new[0] = state[0] + action_row_change[action]
            state_new[1] = state[1] + action_col_change[action]

            #put the robot back in grid if it goes out. Consider also the option to give a negative reward
            if state_new[0] < 0:
                state_new[0] = 0
            if state_new[0] >= N:
                state_new[0] = N-1
            if state_new[1] < 0:
                state_new[1] = 0
            if state_new[1] >= M:
                state_new[1] = M-1

            s_index_new = np.ravel_multi_index(state_new,dims=(N,M),order='F')  #conversion in a single index

            ## TODO update Qvalues. Only if is not the first step
            if step > 1:
                # Update weights
                dw = learning_rate * (r_old - Q_old + gamma * Q[action]) * output_old.dot(input_old.T)
                weights += dw
                
            #store variables for sarsa computation in the next step
            output = np.zeros((N_actions,1))
            output[action] = 1
            
            #update variables
            output_old = output
            input_old = input_vector
            Q_old = Q[action]
            r_old = 0
   
            state[0] = state_new[0]
            state[1] = state_new[1]
            s_index = s_index_new

            ## TODO: check if state is terminal and update the weights consequently
            if s_index == s_end:
                #pass   #pass means doing nothing
                        
                # Update weights for the terminal state
                dw = learning_rate * (R - Q_old) * output_old.dot(input_old.T)
                weights += dw             
                
                pass
    
       
    return learning_curve

In [ ]:
#########    learning_rate    ###############

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nTrials = 500        # should be integer >0
learning_rate = np.linspace(0, 1, 10)
eps = 0.1       # should be real, Greater or Equal to 0; epsilon=0 Greedy, otherwise epsilon-Greedy
gamma = 0.5         # should be real, positive, smaller than 1
repetitions = 50    # number of episodes, should be integer, greater than 0; for statistical reasons

N_actions = 4
N_states = 49

average_steps = np.zeros(10)

# Main loop
for i in np.arange(learning_rate.size):
    
    steps = homing_nn_TD(nTrials, learning_rate[i], eps,gamma)
    average_steps[i] = np.mean(steps)
    print(i)
l = average_steps

#plt.subplots(1, 1)
#plt.plot(learning_rate, average_steps)
#plt.show()

In [ ]:
%matplotlib inline
plt.plot(l1, average_steps)

In [ ]:
#########    eps   ###############

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nTrials = 500        # should be integer >0
eps = np.linspace(0, 1, 10)
learning_rate = 0.5       # should be real, Greater or Equal to 0; epsilon=0 Greedy, otherwise epsilon-Greedy
gamma = 0.5         # should be real, positive, smaller than 1
repetitions = 50    # number of episodes, should be integer, greater than 0; for statistical reasons

N_actions = 4
N_states = 49

average_steps = np.zeros(10)

# Main loop
for i in np.arange(10):
    
    steps = homing_nn_TD(nTrials, learning_rate, eps[i],gamma)
    average_steps[i] = np.mean(steps)
    #print(i)
e = average_steps

#plt.subplots(1, 1)
#plt.plot(eps, average_steps)
#plt.show()

In [ ]:
%matplotlib inline
plt.plot(e1, average_steps)

In [ ]:
#########    gamma  ###############

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nTrials = 500        # should be integer >0
gamma = np.linspace(0, 1, 10)
learning_rate = 0.5       # should be real, Greater or Equal to 0; epsilon=0 Greedy, otherwise epsilon-Greedy
eps = 0.5         # should be real, positive, smaller than 1


N_actions = 4
N_states = 49

average_steps = np.zeros(10)

# Main loop
for i in np.arange(10):
    
    steps = homing_nn_TD(nTrials,learning_rate, eps, gamma[i])
    average_steps[i] = np.mean(steps)
    print(i)
g = average_steps

#plt.subplots(1, 1)
#plt.plot(gamma, average_steps)
#plt.show()

In [ ]:
%matplotlib inline
plt.plot(e1, average_steps)

In [ ]:
import pylab as plt
import matplotlib.patches as mpatches

plt.figure(figsize=(15,6))
x = np.linspace(0, 1,10) 
y = [ l, e, g ]
labels=['Learning rate', 'Epsilon', 'Gamma']
colors=['r','g','b']

# loop over data, labels and colors
for i in range(len(y)):
    plt.plot(x,y[i],'-',color=colors[i],label=labels[i])


plt.legend(fontsize=18)
plt.title('Average steps over parameters ',fontsize=18) 
plt.xlabel('parameters',fontsize = 18)
plt.ylabel('Average steps',fontsize = 18)
plt.ylim([0,50])
plt.show()

In [3]:
##plot average learning curve##
import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np
n_trials = 500
eps = 0.5
gamma = 0.5                                                     #curve full down very quickly when applying high gamma 
learning_rate = 0.5
repetitions = 10


total = np.zeros((repetitions,n_trials))
for i in range(repetitions):
    total[i,:] =  homing_nn_TD(n_trials,learning_rate,eps,gamma)

means = np.mean(total,axis=0)
errors = 2 * np.std(total, axis = 0) / np.sqrt(repetitions)

 
# print ending time caculating time
#plot figures
#plt.title('Exploration steps over each trial with errorbar',fontsize=16)
#plt.xlabel('Trials',fontsize=14)
#plt.ylabel('Steps',fontsize=14)
#plt.ylim([0,60])
#plt.errorbar(np.arange(n_trials),means,errors,color='blue',fmt='-',ecolor='red',elinewidth = 3)
#plt.show()

In [5]:
##plot average learning curve##
import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np
n_trials = 500
eps = 0.01
gamma = 0.99                                                     #curve full down very quickly when applying high gamma 
learning_rate = 0.99
repetitions = 10

total_op = np.zeros((repetitions,n_trials))
for i in range(repetitions):
    total_op[i,:] =  homing_nn_TD(n_trials,learning_rate,eps,gamma)

means_op = np.mean(total_op,axis=0)
errors_op = 2 * np.std(total_op, axis = 0) / np.sqrt(repetitions)

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(111)
# add labels
ax.errorbar(np.arange(n_trials),means, errors, 0, fmt='-',ecolor='r', elinewidth = 3)
ln1=ax.plot(np.arange(n_trials),means,'b',linewidth = 1.5, label='unoptimized')
ax.errorbar(np.arange(n_trials),means_op, errors_op,fmt='-',ecolor='g',elinewidth = 3)
ln2=ax.plot(np.arange(n_trials),means_op,'y',linewidth = 1.5, label='optimized')

ln=ln1+ln2
labs=[i.get_label() for i in ln]
ax.legend(ln,labs)

plt.title('Average steps over each trial with eligibilty trace and without',fontsize=18)
plt.xlabel('Trial',fontsize = 16)
plt.ylabel('Average Steps',fontsize = 16)
plt.ylim([0,60])
plt.show()